Ce dataset contient les statistiques d’embauche pour une entreprise telles que l’expérience du candidat, sa note au test écrit et sa note pour l’entretien personnel. Sur la base de ces 3 facteurs, les ressources humaines doivent décider du salaire. 
Objectif : En se basant sur ces données, nous allons créer un modèle d’apprentissage automatique pour le service des ressources humaines, qui va les aider à déterminer les salaires des futurs candidats.

In [2]:
pip install word2number

  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built word2number


In [5]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
# Importons les librairies nécessaires

import pandas as pd
import numpy as np
from sklearn import linear_model
from word2number import w2n

Chargeons le dataset

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Download du fichier
id = '1o8TI_dHkhg-LLCgrdwO27A6gm1dZGyuh'
downloaded = drive.CreateFile({'id':'1o8TI_dHkhg-LLCgrdwO27A6gm1dZGyuh'})
downloaded.GetContentFile('hiring.csv')

In [0]:
# Import du fichier csv dans Python
data = pd.read_csv('hiring.csv', sep=',')

In [13]:
data

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,NaN,8.0,9,50000
1,NaN,8.0,6,45000
2,five,6.0,7,60000
3,two,10.0,10,65000
4,seven,9.0,6,70000
5,three,7.0,10,62000
6,ten,NaN,7,72000
7,eleven,7.0,8,80000


On constate que nous avons des données manquantes (NaN) aussi bien dans la colonne 'experience' que dans la colonne 'test_score'. Gérons-les.

### Gestion des données manquantes

Dans la colonne 'experience', nous allons remplacer les valeurs non connues par zéro.

In [14]:
data.experience = data.experience.fillna('zero')
data

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,zero,8.0,9,50000
1,zero,8.0,6,45000
2,five,6.0,7,60000
3,two,10.0,10,65000
4,seven,9.0,6,70000
5,three,7.0,10,62000
6,ten,NaN,7,72000
7,eleven,7.0,8,80000


Remplaçons la valeur non connue de la colonne 'test_score(outof 10)' par la médiane de la colonne

In [15]:
import math
median_test_score = math.floor(data['test_score(out of 10)'].mean())
median_test_score

7

In [16]:
data['test_score(out of 10)'] = data['test_score(out of 10)'].fillna(median_test_score)
data

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,zero,8.0,9,50000
1,zero,8.0,6,45000
2,five,6.0,7,60000
3,two,10.0,10,65000
4,seven,9.0,6,70000
5,three,7.0,10,62000
6,ten,7.0,7,72000
7,eleven,7.0,8,80000


### Transformons les données de la colonne 'experience' en chiffres

In [17]:
data.experience = data.experience.apply(w2n.word_to_num)
data

,experience,test_score(out of 10),interview_score(out of 10),salary($)
0,0,8.0,9,50000
1,0,8.0,6,45000
2,5,6.0,7,60000
3,2,10.0,10,65000
4,7,9.0,6,70000
5,3,7.0,10,62000
6,10,7.0,7,72000
7,11,7.0,8,80000


### Construisons le modèle

In [18]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(data[['experience','test_score(out of 10)','interview_score(out of 10)']],data['salary($)'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Prédisons le salaire d'un candidat ayant 5 ans d'expérience, 6 au score et 7 à l'interview

In [20]:
reg.predict([[5, 6, 7]])

array([57964.2290839])

Un candidat ayant 5 années d'expérience, 6 au score et 7 en entretien doit s'attendre à avoir un savoir de 57964.2290839.